In [1]:
import json
import csv
import pymysql
import time

from collections import defaultdict

### Extracting queries

In [12]:
dbs = ['car_1', 'wine_1', 'student_1', 'inn_1', 'flight_1', 'formula_1']
curr = "formula_1"

def get_queries(database):
    # Open the JSON file and load its contents
    with open(database, "r") as file:
        data = json.load(file)

    query_list = []
    temp = []
    for i in range(len(data)):
        variables = data[i]['variables']
        sql_queries = data[i]['sql']

        # replacing each occurrence of a particular variable in any of the queries
        for p, sql_query in enumerate(sql_queries):
            sql_queries[p] = sql_query
            for var in variables:
                sql_queries[p] = sql_queries[p].replace(var['name'], var['example'])
            if data[i]["sentences"][0]["database"] == curr:
                query_list.append(sql_queries[p])
                temp.append(data[i]["sentences"][0]["database"])


    
    cnt = defaultdict(int)
    for i in range(len(temp)):
        cnt[temp[i]] += 1

    print(cnt)
    return query_list

In [13]:
all_queries = []

# all_queries.extend(get_queries("./atis.json"))
queries = dict()

import os
for filename in os.listdir('.'):
    if filename.endswith('spider.json'):
        all_queries.extend(get_queries(f'./{filename}'))
        queries[filename] = get_queries(f'./{filename}')

        

print(len(all_queries))

defaultdict(<class 'int'>, {'formula_1': 40})
defaultdict(<class 'int'>, {'formula_1': 40})
40


In [14]:
len(all_queries)
for filename in os.listdir('.'):
    if filename.endswith('spider.json'):
        print(f'{filename}: {len(queries[filename])}')

spider.json: 40


In [15]:
s = set()
with open('./spider.json', 'r') as f:
    data = json.load(f)

for i in range(len(data)):
    sentences = data[i]["sentences"]
    for sentence in sentences:
        s.add(sentence["database"])

print(s)
print(len(s))

{'customers_and_addresses', 'manufacturer', 'medicine_enzyme_interaction', 'climbing', 'customers_campaigns_ecommerce', 'geo', 'department_store', 'scholar', 'riding_club', 'device', 'club_1', 'department_management', 'cre_Theme_park', 'wedding', 'behavior_monitoring', 'journal_committee', 'train_station', 'game_injury', 'flight_2', 'product_catalog', 'company_employee', 'county_public_safety', 'phone_market', 'hr_1', 'assets_maintenance', 'sakila_1', 'race_track', 'tracking_orders', 'small_bank_1', 'college_1', 'orchestra', 'hospital_1', 'performance_attendance', 'customer_complaints', 'company_1', 'pets_1', 'tracking_software_problems', 'workshop_paper', 'company_office', 'apartment_rentals', 'machine_repair', 'flight_1', 'music_2', 'customers_card_transactions', 'local_govt_and_lot', 'shop_membership', 'party_people', 'network_1', 'wta_1', 'election', 'document_management', 'poker_player', 'scientist_1', 'museum_visit', 'products_for_hire', 'movie_1', 'news_report', 'game_1', 'schoo

In [16]:
with open('./data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['query'])
    for query in all_queries:
        writer.writerow([query])

### Obtaining execution time

At this point, we have all the queries. Now, we write a script to run the queries one by one and store their execution times in the CSV file.

In [17]:
# Establish connection to the database
# connection = pymysql.connect(
#     host='localhost',
#     user='root',
#     password=''
# )

import sqlite3
sqlite_file = f'/mnt/d/Documents/UCSD/Courses/CSE 291 Virt/temp/data/{curr}.sqlite'

connection = sqlite3.connect(sqlite_file)

cursor = connection.cursor()

In [18]:
cursor.execute(f'CREATE DATABASE IF NOT EXISTS ')

OperationalError: near "DATABASE": syntax error

In [6]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
print(tables)

OperationalError: near ".": syntax error

In [19]:
cursor.connection.commit

<function Connection.commit()>

In [21]:
with open('./data.csv', newline='') as csvfile:
    query_reader = csv.reader(csvfile)
    rows = list(query_reader)

# Add header for the "running time" column
rows[0].append('running time')

for row in rows[1:]:
    start_time = time.time()
    sql_query = row[0]
    cursor.execute(sql_query)
    cursor.connection.commit()
    end_time = time.time()
    running_time = (end_time - start_time) * 1000
    row.append(str(running_time) + ' ms')

In [36]:
cursor.close()

In [37]:
with open('./queries_with_running_time.csv', 'w', newline='') as csvfile:
    query_writer = csv.writer(csvfile)
    query_writer.writerows(rows)